**Pytorch BERT baseline**

In this version, I convert https://www.kaggle.com/akensert/bert-base-tf2-0-minimalistic into pytorch version

**Please upvote the kernel if you find it helpful**

### Install HuggingFace transformers & sacremoses dependency

As we are not allowed to use internet I've created required datasets and commands to setup Hugging Face Transformers setup in offline mode. You can find the required github codebases in the datasets.

* sacremoses dependency - https://www.kaggle.com/axel81/sacremoses
* transformers - https://www.kaggle.com/axel81/transformers

In [1]:
# !pip install ./sacremoses/sacremoses-master/
# !pip install ./transformers/transformers-master/
STRIDE = 1
def is_jupyter():
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return True
        
    except:
        return False

### Required Imports

I've added imports that will be used in training too

In [2]:
import re
from random import shuffle as shfl
from auc import MyAUCCallback
from sklearn.utils import shuffle
from datetime import datetime
import pandas as pd
pd.set_option('display.max.columns', 500)
import numpy as np
import os
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-10.1/lib64'
import matplotlib.pyplot as plt
import gc
from shutil import copyfile
from catalyst.dl import SupervisedRunner, AlchemyLogger, CriterionCallback
from catalyst.dl.callbacks.metrics import AUCCallback
from torch.utils.data import DataLoader, SubsetRandomSampler,Dataset
batch_size = 6
token = "d1dd16f08d518293bcbeddd313b49aa4"
DATA_DIR = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'

/home/kb/jig_env/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [3]:
if os.uname()[1] == 'kb-Z370P-D3':
    # desktop
    LOG_PATH = '/media/ssd/logs/jigsaw'
    SERVER = False
    print('Working on desktop')
elif os.uname()[1] == 'kb-server':
    # server
    LOG_PATH = '/home/kb/logs/jigsaw'
    SERVER = True
    print('Working on server')
else:
    raise Exception('which hostname???')
    

Working on desktop


In [4]:
def clean(text):
    text = text.fillna("fillna")#.str.lower()
    text = text.map(lambda x: re.sub('\\n',' ',str(x)))
    text = text.map(lambda x: re.sub("\[\[User.*",'',str(x)))
    text = text.map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
    text = text.map(lambda x: re.sub("\(http://.*?\s\(http://.*\)",'',str(x)))
    return text

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

def clean_text(x):
    x = str(x).replace("\n","")
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def clean_data(df, columns: list):
    for col in columns:
        pass
        df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = clean(df[col])
        df[col] = df[col].apply(lambda x: clean_text(x)) 
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))
# #         df[col] = df[col].apply(lambda x: handle_contractions(x))  
#         df[col] = df[col].apply(lambda x: fix_quote(x))   
    
    return df

<>:4: DeprecationWarning:

invalid escape sequence \[

<>:5: DeprecationWarning:

invalid escape sequence \d

<>:6: DeprecationWarning:

invalid escape sequence \(

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



In [5]:
df_train_toxic = pd.read_csv(DATA_DIR+'jigsaw-toxic-comment-train.csv')
df_train_toxic = clean_data(df_train_toxic, ['comment_text'])
df_train_toxic.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,D ' aww ! He matches this background colour I...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man , I ' m really not trying to edit war...",0,0,0,0,0,0
3,0001b41b1c6bb37e,""" More I can ' t make any real suggestions o...",0,0,0,0,0,0
4,0001d958c54c6e35,"You , sir , are my hero . Any chance you re...",0,0,0,0,0,0


In [6]:
df_train_bias = pd.read_csv(DATA_DIR + 'jigsaw-unintended-bias-train.csv')
df_train_bias.head()

,id,comment_text,toxic,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,Is this something I'll be able to install on m...,0.000000,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,haha you guys are a bunch of losers.,0.893617,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [7]:
# len(df_train_toxic[df_train_toxic['toxic']==1]),len(df_train_toxic[df_train_toxic['toxic']==0])
# len(df_train_bias[df_train_bias['toxic']<0.1])/7,len(df_train_bias[df_train_bias['toxic']>0.80])
# len(df_train_bias[df_train_bias['toxic']==1]),len(df_train_bias[df_train_bias['toxic']==0])
len(df_train_bias[df_train_bias['toxic']==0])/20,len(df_train_bias[df_train_bias['toxic']>0.8])

(66651.75, 25124)

In [8]:
df_train_bias_pos = df_train_bias[df_train_bias['toxic']>0.8].reset_index(drop=True)
df_train_bias_pos['toxic'] = 1
df_train_bias_neg = shuffle(df_train_bias[df_train_bias['toxic']<0.1].reset_index(drop=True)).reset_index(drop=True).iloc[::20]
df_train_bias_neg['toxic'] = 0
df_train_bias =  df_train_bias_pos.append(df_train_bias_neg).reset_index(drop=True)
df_train_bias = clean_data(df_train_bias, ['comment_text'])


In [9]:
# df_valid = shuffle(pd.read_csv(DATA_DIR + 'validation.csv'))
# df_valid = clean_data(df_valid, ['comment_text'])
# df_valid.head()

In [10]:
df_valid_en = shuffle(pd.read_csv(DATA_DIR + 'validation_en.csv'))
df_valid_en = df_valid_en.drop(['comment_text'],axis=1).rename(columns={'comment_text_en':'comment_text'})
df_valid_en = clean_data(df_valid_en, ['comment_text'])
df_valid_en.head()

,id,lang,toxic,comment_text
6291,6291,tr,0,The world ' s free encyclopedia aimed at disse...
2010,2010,it,0,"Spider , you should shut up because you are a..."
7648,7648,it,0,And an activist ( but not peace ) is also a ...
5950,5950,tr,0,Jedistarlogo . gif license suspicion # # px ...
2169,2169,it,0,I apologize . I ' m not being precious but in...


In [11]:
df_valid_en.groupby(['lang', 'toxic']).count()

id  comment_text
lang toxic                    
es   0      2078          2078
     1       422           422
it   0      2012          2012
     1       488           488
tr   0      2680          2680
     1       320           320

In [12]:
# df_test = pd.read_csv(DATA_DIR + 'test.csv')
# df_test.head()

In [13]:
# df_test.groupby(['lang']).count()

In [14]:
# len(df_test)

In [15]:
df_sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
df_sub.head()

,id,toxic
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [16]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [17]:
target_column = 'toxic'

### Define dataset

In [18]:
# 7from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import time
from torch.optim import lr_scheduler

import torch
from tqdm import tqdm
#import torch.utils.data as data
from torchvision import datasets, models, transforms
from transformers import *
import random
from math import floor, ceil
from sklearn.model_selection import GroupKFold

MAX_LEN = 512
SEP_TOKEN_ID = 102

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True):
        
            
        self.df = df
        if train_mode:
            self.labels = df.toxic.values
            self.toxic_inds = np.where(self.labels==1)[0]
            self.normal_inds = np.where(self.labels==0)[0]            
            
            
            print(f'Here is {len(self.labels)} samples, {len(self.toxic_inds)} samples and {len(self.normal_inds)} samples')
            print(f'Class balance is {len(self.toxic_inds)/len(self.labels):.2f}')
            
        self.train_mode = train_mode
        self.labeled = labeled
#         self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#         self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#         self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
#         self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
#         self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')
#         self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased',
#                                                                 do_lower_case=False,
#                                                                 do_basic_tokenize=True,
#                                                                 never_split=None,
#                                                                 unk_token='[UNK]',
#                                                                 sep_token='[SEP]',
#                                                                 pad_token='[PAD]',
#                                                                 cls_token='[CLS]',
#                                                                 mask_token='[MASK]',
#                                                                 tokenize_chinese_chars=True,)
        #distil

    def __getitem__(self, index):
        row = self.df.iloc[index]
        token_ids = self.get_token_ids(row)
        
        if self.labeled:
            labels = self.get_label(row)
            return {'features': token_ids, 'targets': labels}

        else:
            return {'features': token_ids}

    def __len__(self):
        return len(self.df)

    def trim_input(self, text, max_sequence_length=MAX_LEN):
        t = self.tokenizer.tokenize(text)
        t_len = len(t)

        if t_len + 2 > max_sequence_length:

            t_new_len = int(max_sequence_length) - 2

            t = t[:t_new_len]

        return t
        
    def get_token_ids(self, row):
        token_ids = self.tokenizer.encode(row.comment_text, max_length=MAX_LEN)
#         print(token_ids)
#         print(token_ids + [0] * (MAX_LEN - len(token_ids)))
        if len(token_ids) < MAX_LEN:
            ids = torch.tensor(token_ids + [0] * (MAX_LEN - len(token_ids)))
        else:
            ids = torch.tensor(token_ids[:MAX_LEN])
#         print(ids.shape)
        
#         t_tokens = self.trim_input(row.comment_text)

# #         tokens = ['[CLS]'] + t_tokens  + ['[SEP]']+ t_tokens[-1::-1]+ ['[SEP]']
#         tokens = ['[CLS]'] + t_tokens  + ['[SEP]']
#         token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        
#         if len(token_ids) < MAX_LEN:
#             token_ids += [0] * (MAX_LEN - len(token_ids))
            
#         ids = torch.tensor(token_ids)
#         print(ids.shape, torch.tensor(token_ids).shape)
#         print(torch.tensor(token_ids))
#         print(len(ids))
        return ids

    def get_label(self, row):
#         label = torch.tensor(row[target_column].astype(np.long))
        label = np.round(row[target_column])
        return torch.tensor([1-label, label]).float()
    
    def collate_fn(self, batch):
        token_ids = torch.stack([x[0] for x in batch])

        if self.labeled:
            labels = torch.stack([x[1] for x in batch])
            return {'features': token_ids, 'targets': labels}
        else:
            return {'features': token_ids}
        
 

/home/kb/jig_env/lib/python3.7/site-packages/torchvision/extension.py:11: ResourceWarning:

unclosed file <_io.BufferedReader name='/home/kb/jig_env/lib/python3.7/site-packages/torchvision/_C.so'>



## Build Model

In [19]:
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F


class QuestModel(nn.Module):
    def __init__(self, n_classes=2):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
        
#         self.bert_model = BertModel.from_pretrained('bert-base-uncased') 
#         self.bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         self.bert_model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
#         self.bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.bert_model = XLMRobertaModel.from_pretrained('xlm-roberta-base', 
                                                          output_hidden_states=False, 
                                                          output_attentions=False)
#         self.bert_model = XLMRobertaModel.from_pretrained('xlm-roberta-large', 
#                                                           output_hidden_states=False, 
#                                                           output_attentions=False)
#         self.bert_model = DistilBertModel.from_pretrained('distilbert-base-cased')
    
        self.fc = nn.Linear(768, n_classes)
#         self.fc = nn.Linear(1024, n_classes)

    def forward(self, ids):
#         attention_mask = (ids > 0)
#         print(ids.shape)
        layers = self.bert_model(input_ids=ids)#, attention_mask=attention_mask)
#         print(layers[0].shape)
#         out = F.dropout(layers[0][:, 0, :], p=0.2, training=self.training)
#         print(layers[0].shape)
#         print([l.shape for l in layers])
#         out = F.dropout(layers[-1][:, 0, :], p=0.35, training=self.training)
        out = F.dropout(layers[0][:, 0, :], p=0.35, training=self.training)
        logit = self.fc(out)#.unsqueeze(1)
        return logit #, 'for_auc': logit[:, 1]}#[:,1]
    


In [20]:
# bert_model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

In [21]:
class BalancedSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset):
       
        self.toxic_inds = dataset.toxic_inds.copy()
        self.normal_inds = dataset.normal_inds.copy()
        
        self.num_samples = 2*min(len(self.toxic_inds), len(self.normal_inds))
        
        shfl(self.toxic_inds)
        shfl(self.normal_inds)
        
        self.inds = []
        for i in range(min(len(self.toxic_inds), len(self.normal_inds))):
            self.inds.append(self.normal_inds[i%len(self.normal_inds)])
            self.inds.append(self.toxic_inds[i%len(self.toxic_inds)])

    def __iter__(self):
        #print ('\tcalling Sampler:__iter__')
        return iter(self.inds)

    def __len__(self):
        #print ('\tcalling Sampler:__len__')
        return self.num_samples




In [22]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
        callback_get_label func: a callback-like function which takes two arguments - dataset and index
    """

    def __init__(self, dataset, indices=None, num_samples=None, callback_get_label=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices

        # define custom callback
        self.callback_get_label = callback_get_label

        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        if self.callback_get_label:
            return self.callback_get_label(dataset, idx)
        else:
            dataset.labels[idx]
#             raise NotImplementedError
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [23]:
dataframes = {
    'bias': df_train_bias,
    'toxic': df_train_toxic,
    'valid': df_valid_en,
#     'valid2': df_test
}
def get_loaders(train_pt='bias', valid_pt='valid', test_pt=None, to_balance=True, shuffle_before=True):
    if SERVER:
        workers=1
    else:
        workers = 6
    
    if isinstance(train_pt, list):
        df_train = dataframes[train_pt[0]][['comment_text', 'toxic']]
        for pt in train_pt[1:]:
            df_train = df_train.append(dataframes[pt][['comment_text', 'toxic']]).reset_index(drop=True)
    else:
        df_train = dataframes[train_pt][['comment_text', 'toxic']]
    
    if shuffle_before:
        df_train = shuffle(df_train).reset_index(drop=True)
    
    
#     train_dataset = QuestDataset(df_train.iloc[::STRIDE], train_mode=True)
    
    if STRIDE == 1:
        train_dataset = QuestDataset(df_train.iloc[::STRIDE], train_mode=True)
    elif STRIDE<=10:
        df_train_pos = df_train[df_train[target_column]==1].reset_index(drop=True)
        df_train_neg = df_train[df_train[target_column]==0].reset_index(drop=True).iloc[::STRIDE]
        train_dataset = QuestDataset(df_train_pos.append(df_train_neg).reset_index(drop=True), train_mode=True)
    else:
        df_train_pos = df_train[df_train[target_column]==1].reset_index(drop=True).iloc[::int(STRIDE/10)]
        df_train_neg = df_train[df_train[target_column]==0].reset_index(drop=True).iloc[::STRIDE]
        train_dataset = QuestDataset(df_train_pos.append(df_train_neg).reset_index(drop=True), train_mode=True)
        
        
    
    valid_dataset = QuestDataset(dataframes[valid_pt], train_mode=False)
    if test_pt != None:
        test_dataset = QuestDataset(dataframes[test_pt].iloc[::STRIDE], train_mode=False)
    
    train_loader = DataLoader(
        train_dataset,
        num_workers=workers,
        sampler=BalancedSampler(train_dataset) if to_balance else None,#ImbalancedDatasetSampler(train_dataset) if to_balance else None,
        batch_size=batch_size,
    )
    valid_loader = DataLoader(
        valid_dataset,
        num_workers=workers,
        batch_size=batch_size,
    )    
    if test_pt != None:
        test_loader = DataLoader(
            test_dataset,
            num_workers=workers,
            batch_size=batch_size,
        )
       
    loaders = {}
    loaders['train'] = train_loader
    
    loaders['valid'] = valid_loader
    
    if test_pt != None:
        loaders['valid2'] = test_loader
    
    for i in ['es', 'it', 'tr']:
        df = dataframes[valid_pt]
        df = df[df['lang']==i]

        loaders['valid_'+ i] = DataLoader(
            QuestDataset(df, train_mode=False),
            num_workers=workers,
            batch_size=batch_size,
        )
    
    
    return loaders


In [24]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [25]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [ ]:
# CHECKPOINT = './checkpoints/from_v4.pth'
project = "jigsaw_v8_xlm_roberta_base"
num_epochs = 300

group = datetime.now().strftime("%m_%d_%Y__%H_%M_%S")

    
if SERVER:
    group = f'{group}_server'
    
if STRIDE > 1:
    group = f'{group}_str{STRIDE}'

variants = {
    'learn_both':{
        'train': ['bias', 'toxic'],
        'valid': 'valid',
#         'valid2': 'valid'
    },
#     'learn_toxic':{
#         'train': 'toxic',
#         'valid': 'valid',
# #         'valid2': 'bias',
#     },
#     'learn_bias':{
#         'train': 'bias',
#         'valid': 'valid',
#         'valid2': 'toxic',
#     },
    
    
}
gradient_accumulation_steps = 1

lr = 3e-5#0.0001
group += f'_lr{lr}'

group = group.replace('.', '')

runner = SupervisedRunner(input_key=('features'), input_target_key=('targets'), output_key=('logits'))

for experiment in variants.keys():
    torch.cuda.empty_cache()
    
    logdir = f"{LOG_PATH}/{project}/{group}/{experiment}"

    model = QuestModel(2)
#     checkpoint = torch.load(CHECKPOINT)#, map_location=device)   

#     model.load_state_dict(checkpoint['model_state_dict'])
#     del checkpoint
#     model = model.to(device)

    # model runner
    if 'valid2' in variants[experiment].keys():
        test_var= variants[experiment]['valid2']
    else:
        test_var=None

    loaders = get_loaders(variants[experiment]['train'], 
                          variants[experiment]['valid'],
                          test_var, to_balance=True)
            
    
    t_total = len(loaders['train'])//gradient_accumulation_steps*num_epochs
    warmup_proportion = 0.01
    num_warmup_steps = t_total * warmup_proportion
    
#     criterion = torch.nn.BCEWithLogitsLoss()
    criterion = FocalLoss(alpha=0.2, gamma=1.5, logits=True, reduce=True)
#     criterion = torch.nn.CrossEntropyLoss()
#     criterion = torch.nn.BCELoss()
    optimizer = AdamW(model.parameters(), lr = lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total) 
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.25)
    print(f'----------------Experiment: {experiment}')

    runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        loaders=loaders,
        logdir=logdir,
        num_epochs=num_epochs,
        verbose=True,
        distributed=False if is_jupyter() else True,
        callbacks=[
            AlchemyLogger(
                    token=token, # your Alchemy token
                    project=project,
                    experiment=experiment,
                    group=group,
                ),
            MyAUCCallback()
#             AUCCallback(input_key = 'targets',
#                                 output_key = 'logits',
#                                 prefix = 'auc',
#                                 class_names = None,
#                                 num_classes = 2,
#                                 activation = 'Sigmoid',)
        ],
        main_metric='auc/_mean',
        minimize_metric=False,
#         fp16={"opt_level": "O1"}

    )

Here is 315403 samples, 46508 samples and 268895 samples
Class balance is 0.15
----------------Experiment: learn_both
1/300 * Epoch (train):   0% 1/15503 [00:00<2:25:05,  1.78it/s, loss=0.050]

/home/kb/jig_env/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning:

size_average and reduce args will be deprecated, please use reduction='none' instead.



1/300 * Epoch (train):  46% 7193/15503 [30:14<34:45,  3.99it/s, loss=0.085]  

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connectionpool.py", line 376, in _make_request
    self._validate_conn(conn)
  File "/home/kb/jig_env/lib/python3.7/site-packages/urllib3/connectionpool.py", line 994, in _validate_c

1/300 * Epoch (train): 100% 15503/15503 [1:05:01<00:00,  3.97it/s, loss=0.052]
1/300 * Epoch (valid): 100% 1334/1334 [01:37<00:00, 13.63it/s, loss=0.025]
1/300 * Epoch (valid_es): 100% 417/417 [00:30<00:00, 13.51it/s, loss=0.041]
1/300 * Epoch (valid_it): 100% 417/417 [00:30<00:00, 13.52it/s, loss=0.025]
1/300 * Epoch (valid_tr): 100% 500/500 [00:36<00:00, 13.54it/s, loss=0.025]
[2020-04-27 01:14:38,451] 
1/300 * Epoch 1 (_base): lr=6.450e-10 | momentum=0.9000
1/300 * Epoch 1 (train): auc/_mean=0.5049 | auc/class_0=0.5049 | loss=0.0604
1/300 * Epoch 1 (valid): auc/_mean=0.4635 | auc/class_0=0.4635 | es_auc/_mean=0.4730 | es_auc/class_0=0.4730 | es_loss=0.0356 | it_auc/_mean=0.4751 | it_auc/class_0=0.4751 | it_loss=0.0372 | loss=0.0346 | tr_auc/_mean=0.4889 | tr_auc/class_0=0.4889 | tr_loss=0.0316
1/300 * Epoch 1 (valid_es): auc/_mean=0.4730 | auc/class_0=0.4730 | loss=0.0356
1/300 * Epoch 1 (valid_it): auc/_mean=0.4751 | auc/class_0=0.4751 | loss=0.0372
1/300 * Epoch 1 (valid_tr): auc/

...

2/300 * Epoch (train): 100% 15503/15503 [1:05:02<00:00,  3.97it/s, loss=0.068]
2/300 * Epoch (valid): 100% 1334/1334 [01:38<00:00, 13.58it/s, loss=0.025]
2/300 * Epoch (valid_es): 100% 417/417 [00:30<00:00, 13.48it/s, loss=0.041]
2/300 * Epoch (valid_it): 100% 417/417 [00:30<00:00, 13.48it/s, loss=0.025]
2/300 * Epoch (valid_tr): 100% 500/500 [00:37<00:00, 13.51it/s, loss=0.025]
[2020-04-27 02:23:30,595] 
2/300 * Epoch 2 (_base): lr=1.290e-09 | momentum=0.9000
2/300 * Epoch 2 (train): auc/_mean=0.5045 | auc/class_0=0.5045 | loss=0.0604
2/300 * Epoch 2 (valid): auc/_mean=0.4639 | auc/class_0=0.4639 | es_auc/_mean=0.4731 | es_auc/class_0=0.4731 | es_loss=0.0356 | it_auc/_mean=0.4754 | it_auc/class_0=0.4754 | it_loss=0.0373 | loss=0.0347 | tr_auc/_mean=0.4895 | tr_auc/class_0=0.4895 | tr_loss=0.0317
2/300 * Epoch 2 (valid_es): auc/_mean=0.4731 | auc/class_0=0.4731 | loss=0.0356
2/300 * Epoch 2 (valid_it): auc/_mean=0.4754 | auc/class_0=0.4754 | loss=0.0373
2/300 * Epoch 2 (valid_tr): auc/

10/300 * Epoch (train): 100% 15503/15503 [1:05:44<00:00,  3.93it/s, loss=0.071]
10/300 * Epoch (valid): 100% 1334/1334 [01:39<00:00, 13.41it/s, loss=0.030]
10/300 * Epoch (valid_es): 100% 417/417 [00:31<00:00, 13.29it/s, loss=0.042]
10/300 * Epoch (valid_it): 100% 417/417 [00:31<00:00, 13.29it/s, loss=0.030]
10/300 * Epoch (valid_tr): 100% 500/500 [00:37<00:00, 13.30it/s, loss=0.030]
[2020-04-27 11:41:47,665] 
10/300 * Epoch 10 (_base): lr=6.450e-09 | momentum=0.9000
10/300 * Epoch 10 (train): auc/_mean=0.5050 | auc/class_0=0.5050 | loss=0.0575
10/300 * Epoch 10 (valid): auc/_mean=0.4907 | auc/class_0=0.4907 | es_auc/_mean=0.4979 | es_auc/class_0=0.4979 | es_loss=0.0378 | it_auc/_mean=0.4960 | it_auc/class_0=0.4960 | it_loss=0.0390 | loss=0.0371 | tr_auc/_mean=0.5237 | tr_auc/class_0=0.5237 | tr_loss=0.0348
10/300 * Epoch 10 (valid_es): auc/_mean=0.4979 | auc/class_0=0.4979 | loss=0.0378
10/300 * Epoch 10 (valid_it): auc/_mean=0.4960 | auc/class_0=0.4960 | loss=0.0390
10/300 * Epoch 10

18/300 * Epoch (train): 100% 15503/15503 [1:05:48<00:00,  3.93it/s, loss=0.052]
18/300 * Epoch (valid): 100% 1334/1334 [01:39<00:00, 13.40it/s, loss=0.041]
18/300 * Epoch (valid_es): 100% 417/417 [00:31<00:00, 13.29it/s, loss=0.045]
18/300 * Epoch (valid_it): 100% 417/417 [00:31<00:00, 13.27it/s, loss=0.044]
18/300 * Epoch (valid_tr): 100% 500/500 [00:37<00:00, 13.31it/s, loss=0.037]
[2020-04-27 21:01:13,429] 
18/300 * Epoch 18 (_base): lr=1.161e-08 | momentum=0.9000
18/300 * Epoch 18 (train): auc/_mean=0.5346 | auc/class_0=0.5346 | loss=0.0545
18/300 * Epoch 18 (valid): auc/_mean=0.5196 | auc/class_0=0.5196 | es_auc/_mean=0.5420 | es_auc/class_0=0.5420 | es_loss=0.0405 | it_auc/_mean=0.5412 | it_auc/class_0=0.5412 | it_loss=0.0411 | loss=0.0398 | tr_auc/_mean=0.4688 | tr_auc/class_0=0.4688 | tr_loss=0.0381
18/300 * Epoch 18 (valid_es): auc/_mean=0.5420 | auc/class_0=0.5420 | loss=0.0405
18/300 * Epoch 18 (valid_it): auc/_mean=0.5412 | auc/class_0=0.5412 | loss=0.0411
18/300 * Epoch 18

26/300 * Epoch (train): 100% 15503/15503 [1:05:26<00:00,  3.95it/s, loss=0.050]
26/300 * Epoch (valid): 100% 1334/1334 [01:39<00:00, 13.45it/s, loss=0.036]
26/300 * Epoch (valid_es): 100% 417/417 [00:31<00:00, 13.35it/s, loss=0.042]
26/300 * Epoch (valid_it): 100% 417/417 [00:31<00:00, 13.34it/s, loss=0.032]
26/300 * Epoch (valid_tr): 100% 500/500 [00:37<00:00, 13.36it/s, loss=0.038]
[2020-04-28 06:15:09,084] 
26/300 * Epoch 26 (_base): lr=1.677e-08 | momentum=0.9000
26/300 * Epoch 26 (train): auc/_mean=0.5476 | auc/class_0=0.5476 | loss=0.0531
26/300 * Epoch 26 (valid): auc/_mean=0.4706 | auc/class_0=0.4706 | es_auc/_mean=0.4999 | es_auc/class_0=0.4999 | es_loss=0.0399 | it_auc/_mean=0.4601 | it_auc/class_0=0.4601 | it_loss=0.0411 | loss=0.0395 | tr_auc/_mean=0.4930 | tr_auc/class_0=0.4930 | tr_loss=0.0379
26/300 * Epoch 26 (valid_es): auc/_mean=0.4999 | auc/class_0=0.4999 | loss=0.0399
26/300 * Epoch 26 (valid_it): auc/_mean=0.4601 | auc/class_0=0.4601 | loss=0.0411
26/300 * Epoch 26

34/300 * Epoch (train): 100% 15503/15503 [1:05:40<00:00,  3.93it/s, loss=0.051]
34/300 * Epoch (valid): 100% 1334/1334 [01:39<00:00, 13.42it/s, loss=0.037]
34/300 * Epoch (valid_es): 100% 417/417 [00:31<00:00, 13.30it/s, loss=0.044]
34/300 * Epoch (valid_it): 100% 417/417 [00:31<00:00, 13.31it/s, loss=0.036]
34/300 * Epoch (valid_tr): 100% 500/500 [00:37<00:00, 13.31it/s, loss=0.034]
[2020-04-28 15:29:18,261] 
34/300 * Epoch 34 (_base): lr=2.193e-08 | momentum=0.9000
34/300 * Epoch 34 (train): auc/_mean=0.5682 | auc/class_0=0.5682 | loss=0.0517
34/300 * Epoch 34 (valid): auc/_mean=0.5302 | auc/class_0=0.5302 | es_auc/_mean=0.5556 | es_auc/class_0=0.5556 | es_loss=0.0402 | it_auc/_mean=0.5331 | it_auc/class_0=0.5331 | it_loss=0.0411 | loss=0.0394 | tr_auc/_mean=0.4652 | tr_auc/class_0=0.4652 | tr_loss=0.0374
34/300 * Epoch 34 (valid_es): auc/_mean=0.5556 | auc/class_0=0.5556 | loss=0.0402
34/300 * Epoch 34 (valid_it): auc/_mean=0.5331 | auc/class_0=0.5331 | loss=0.0411
34/300 * Epoch 34

In [ ]:
# DistilBertTokenizer?


#### 